## scikit-learn sample_weight compliance investigative plots

This notebook runs compliance tests on a given scikit-learn estimators. This allows one to plot different features of the resulting predictions and scores with and without weights, so as to check weighted-repeated equivalence.

In [ ]:
import sklearn
import threadpoolctl

# HistGradientBoostingClassifier trashes the OpenMP thread pool on repeated
# small fits.
_ = threadpoolctl.threadpool_limits(limits=1, user_api="openmp")

sklearn.show_versions()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sample_weight_audit import check_weighted_repeated_estimator_fit_equivalence
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier


# Note: strong regularization (low C value) makes LogisticRegression
# predictions sensitive to number of data points: this can be a problem when
# used in conjunction with BaggingClassifier, which resamples for a given
# number of data points, that can be defined either as a fraction of the
# training set or as an absolute number.
# solver = "saga"  # stochastic, but does not converge for small C values.
solver = "lbfgs"  # deterministic and stable on this dataset.
estimator = LogisticRegression(C=1e-3, solver=solver, max_iter=1_000)
# estimator = DecisionTreeClassifier(max_depth=5)

# max_samples = 1.0  # relative number of data points to resample during bagging.
max_samples = 100  # absolute number of data points to resample during bagging.
estimator = BaggingClassifier(estimator, n_estimators=30, max_samples=max_samples)
estimator_name = estimator.__class__.__name__

result = check_weighted_repeated_estimator_fit_equivalence(
    estimator,
    estimator_name,
    test_name="kstest",
    n_stochastic_fits=100,
    n_samples_per_cv_group=300,
    random_state=0,
)

 50%|█████     | 50/100 [00:09<00:09,  5.23it/s]

# Make plots of scores

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

alpha = 0.5
eps = 1e-6
min_score = min(result.scores_repeated.min(), result.scores_weighted.min())
max_score = max(result.scores_repeated.max(), result.scores_weighted.max())
if max_score - min_score < eps:
    min_score -= eps
    max_score += eps

bins_scores = np.linspace(min_score, max_score, 15)
ax.hist(result.scores_repeated, alpha=alpha, bins=bins_scores, label="repeated")
ax.hist(result.scores_weighted, alpha=alpha, bins=bins_scores, label="weighted")
ax.legend()
_ = ax.set_title(f"KS p-value: {result.pvalue:.2g}")

# Make plots of predictions

## Plot the histograms of predictions for some randomly selected test points

In [ ]:
fig, axs = plt.subplots(5, 4, figsize=(14, 12))
n_plots = len(axs.flatten())

rng = np.random.default_rng(0)
sample_indices = rng.choice(
    np.arange(result.predictions_weighted.shape[1]), size=n_plots, replace=False
)

min_prediction = min(
    result.predictions_repeated.min(), result.predictions_weighted.min()
)
max_prediction = max(
    result.predictions_repeated.max(), result.predictions_weighted.max()
)
if max_prediction - min_prediction < eps:
    min_prediction -= eps
    max_prediction += eps
bins_predictions = np.linspace(min_prediction, max_prediction, 30)

for sample_idx, ax in zip(sample_indices, axs.flatten()):
    ax.hist(
        result.predictions_repeated[:, sample_idx].flatten(),
        alpha=alpha,
        bins=bins_predictions,
        label="repeated",
    )
    ax.hist(
        result.predictions_weighted[:, sample_idx].flatten(),
        alpha=alpha,
        bins=bins_predictions,
        label="weighted",
    )

# Show the legend only on the last subplot
_ = ax.legend()


## Zooming into individual samples

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8, 6))

sample_idx = 50
ax.hist(result.predictions_repeated[:, sample_idx].flatten(), alpha=alpha, bins=bins_predictions, label='repeated')
ax.hist(result.predictions_weighted[:, sample_idx].flatten(), alpha=alpha, bins=bins_predictions, label='weighted')
_ = ax.legend()